# Analysing the GDELT events

In [1]:
#!pip install gdelt

In [2]:
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
import datetime as dt
import gdelt as gd
import numpy as np
import pandas as pd

In [3]:
gdelt_client = gd.gdelt()

In [4]:
date = dt.datetime.now() - dt.timedelta(days=1)
date_string = date.strftime('%Y %m %d')
gdelt_events = gdelt_client.Search(date_string, table='events', coverage=True)

In [5]:
gdelt_events['CAMEOCodeDescription'].unique()
gdelt_events.columns

Index(['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'CAMEOCodeDescription', 'EventBaseCode', 'EventRootCode',
       'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
       'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName',
       'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code',
       'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID',
       'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode',
       'Actor2Geo_ADM1Code', 'Actor2Geo_ADM2Code

In [6]:
#gdelt_events.groupby(['Actor1Name', 'ActionGeo_FullName', 'Actor2Name', 'CAMEOCodeDescription']).size().sort_values(ascending=False).reset_index(name='counts')
gdelt_events.groupby(['ActionGeo_FullName', 'CAMEOCodeDescription']).size().sort_values(ascending=False).reset_index(name='counts')

,ActionGeo_FullName,CAMEOCodeDescription,counts
0,"Rome, Lazio, Italy","Consult, not specified below",343
1,"Rome, Lazio, Italy",Express intent to meet or negotiate,259
2,United States,"Consult, not specified below",201
3,"Paris, France (general), France","Consult, not specified below",201
4,"White House, District of Columbia, United States","Consult, not specified below",194
...,...,...,...
33378,"Massachusetts, United States",Make empathetic comment,1
33379,"Massachusetts, United States",Mobilize or increase armed forces,1
33380,"Massachusetts, United States",Provide military protection or peacekeeping,1
33381,"Massachusetts, United States",Sexually assault,1


In [33]:
# Find demonstrations/protests
protests_codes = ['014', '141'] \
    + ['141' + str(index) for index in range(1, 5)] \
    + ['142'] \
    + ['142' + str(index) for index in range(1, 5)] \
    + ['143'] \
    + ['143' + str(index) for index in range(1, 5)] \
    + ['144'] \
    + ['144' + str(index) for index in range(1, 5)] \
    + ['145'] \
    + ['145' + str(index) for index in range(1, 5)]

# Geo types 3:='US city', 4:= 'World city'
geo_types = [3, 4]
gdelt_protests = gdelt_events.query('EventCode in @protests_codes').query('ActionGeo_Type in @geo_types')
gdelt_protests.groupby(['ActionGeo_FullName']).size().sort_values(ascending=False).reset_index(name='counts')

,ActionGeo_FullName,counts
0,"London, London, City of, United Kingdom",49
1,"Paris, France (general), France",38
2,"Khartoum, Al Khartum, Sudan",35
3,"Glasgow, Glasgow City, United Kingdom",35
4,"White House, District of Columbia, United States",34
...,...,...
499,"Langley, Northumberland, United Kingdom",1
500,"Layer-De-La-Haye, Essex, United Kingdom",1
501,"Topkapi, Adiyaman, Turkey",1
502,"Lima, Lima, Peru",1


In [7]:
# Connect to ArcGIS Online anonymously
gis = GIS()

europe_map = gis.map("Europe")
europe_map.basemap = "dark-gray-vector"

In [8]:
WGS84 = 4326
gdelt_spatial = GeoAccessor.from_xy(gdelt_events, x_column='ActionGeo_Long', y_column='ActionGeo_Lat', sr=WGS84)
gdelt_spatial.spatial.plot(map_widget=europe_map, renderer_type='s')
europe_map

MapView(layout=Layout(height='400px', width='100%'))

In [9]:
import algorithm

In [10]:
gdelt_events.groupby(['ActionGeo_FullName']).size().sort_values(ascending=False).reset_index(name='counts')

,ActionGeo_FullName,counts
0,"Gaza, Israel (general), Israel",4456
1,"Jerusalem, Israel (general), Israel",2409
2,United States,1359
3,"Washington, District of Columbia, United States",1160
4,"New York, United States",1095
...,...,...
4973,"Kill Devil Hills, North Carolina, United States",1
4974,"King Saud University, Ar Riya?, Saudi Arabia",1
4975,"Kingdom Of Buganda, Mityana, Uganda",1
4976,"Kingstown, Saint George, Saint Vincent And The...",1


In [26]:
names_values = gdelt_events['ActionGeo_FullName'].dropna()
names_list = names_values.tolist()
algorithm.count(names_list, 'Gaza, Israel (general), Israel')

4456

In [27]:
names_arr = names_values.to_numpy()
algorithm.count(names_arr, 'Gaza, Israel (general), Israel')

4456